# 🎨 AI-Powered Doodle Recognition & Styling - Colab Backend

This notebook provides a complete backend for the AI-Powered Doodle Recognition platform running on Google Colab with GPU acceleration.

## Features:
- 🤖 **MobileNetV2/V3** for efficient doodle recognition
- 🎨 **Stable Diffusion** with ControlNet for image generation
- 🚀 **FastAPI** server with ngrok tunneling
- 📊 **Training pipeline** for custom models
- 🤗 **Hugging Face** integration ready

## Quick Start:
1. Set your `NGROK_AUTH_TOKEN` in Colab Secrets
2. Run all cells in order
3. Use the public URL to connect your frontend

---

## 📦 Setup and Installation

In [ ]:
# Clone the repository if not already available
import os
if not os.path.exists('colab_backend'):
    !git clone https://github.com/RohithCherukuri816/AI-Powered-Creative-Intelligence-Platform.git
    %cd AI-Powered-Creative-Intelligence-Platform
else:
    print("✅ Repository already available")

In [ ]:
# Install all required dependencies
!pip install -q -r colab_backend/requirements.txt
print("✅ Dependencies installed")

In [ ]:
# Check system capabilities
import torch
import sys

print("🖥️  System Information:")
print(f"   Python: {sys.version}")
print(f"   PyTorch: {torch.__version__}")
print(f"   CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
print(f"   CPU Count: {os.cpu_count()}")

## 🤖 MobileNet Training (Optional)

Train a custom MobileNet model on Quick Draw dataset. Skip this section if you want to use the pre-trained model.

In [ ]:
# Import training modules
from colab_backend.mobilenet_trainer import MobileNetTrainer
import matplotlib.pyplot as plt

# Initialize trainer
trainer = MobileNetTrainer(model_name="mobilenetv3_large_100", num_classes=10)  # Using 10 classes for demo
print("✅ Trainer initialized")

In [ ]:
# Download Quick Draw data (subset for demo)
categories = ['car', 'cat', 'dog', 'house', 'tree', 'airplane', 'bicycle', 'bird', 'flower', 'fish']
images, labels, class_names = trainer.download_quickdraw_data(categories=categories, samples_per_class=500)

print(f"✅ Downloaded {len(images)} images across {len(class_names)} categories")
print(f"   Image shape: {images[0].shape}")
print(f"   Categories: {class_names}")

In [ ]:
# Prepare data loaders
trainer.prepare_data(images, labels, batch_size=32)
print("✅ Data loaders prepared")

In [ ]:
# Create and train model
trainer.create_model(pretrained=True)
print("✅ Model created with transfer learning")

# Train for a few epochs (increase for better results)
trainer.train(epochs=5, learning_rate=0.001)
print("✅ Training completed")

In [ ]:
# Evaluate model
accuracy, predictions, targets = trainer.evaluate()
print(f"✅ Test accuracy: {accuracy:.4f}")

# Plot training history
trainer.plot_training_history()

In [ ]:
# Save model for Hugging Face
model_dir = trainer.save_model_for_huggingface("mobilenet_doodle_model")
print(f"✅ Model saved to {model_dir}")

# Optionally upload to Hugging Face Hub
# from huggingface_hub import HfApi
# api = HfApi()
# api.upload_folder(
#     folder_path=model_dir,
#     repo_id="your-username/doodle-recognition-mobilenet",
#     repo_type="model"
# )

## 🚀 Start FastAPI Server

Launch the complete backend server with MobileNet recognition and Stable Diffusion generation.

In [ ]:
# Set environment variables
import os
from google.colab import userdata

# Set Hugging Face token if available
try:
    hf_token = userdata.get('HUGGINGFACE_TOKEN')
    os.environ['HUGGINGFACE_TOKEN'] = hf_token
    print("✅ Hugging Face token set")
except:
    print("⚠️  Hugging Face token not found in secrets")

# Configure CORS for frontend connection
os.environ['CORS_ALLOWED_ORIGINS'] = 'http://localhost:5173,http://localhost:5174'
print("✅ Environment configured")

In [ ]:
# Test the backend components
from colab_backend.mobilenet_recognizer import MobileNetDoodleRecognizer
from colab_backend.processor import ImageProcessor
from PIL import Image
import numpy as np

# Initialize components
recognizer = MobileNetDoodleRecognizer()
processor = ImageProcessor()

print("✅ Backend components initialized")
print(f"   Model info: {recognizer.get_model_info()}")
print(f"   Device: {processor.device}")

In [ ]:
# Setup ngrok tunnel
!pip install -q pyngrok

from pyngrok import ngrok
import subprocess
import sys

# Get ngrok token from Colab Secrets
try:
    ngrok_token = userdata.get('NGROK_AUTH_TOKEN')
    ngrok.set_auth_token(ngrok_token)
    print("✅ Ngrok authenticated")
except:
    print("❌ NGROK_AUTH_TOKEN not found in Colab Secrets!")
    print("   Please add your ngrok token to Colab Secrets with key 'NGROK_AUTH_TOKEN'")
    print("   Get your token from: https://dashboard.ngrok.com/get-started/your-authtoken")
    raise SystemExit("Ngrok token required")

# Create public tunnel
public_url = ngrok.connect(8000, 'http')
print(f"🌐 Public URL: {public_url}")
print(f"📚 API Docs: {public_url}/docs")
print(f"💚 Health Check: {public_url}/health")

In [ ]:
# Launch FastAPI server
print("🚀 Starting FastAPI server...")
print("   The server will run in the background")
print("   Use the public URL above to connect your frontend")
print("   Press Ctrl+C in the terminal to stop the server")

# Start server process
proc = subprocess.Popen([
    sys.executable, 
    '-m', 
    'uvicorn', 
    'colab_backend.app:app', 
    '--host', '0.0.0.0', 
    '--port', '8000',
    '--reload'
])

print(f"✅ Server started with PID: {proc.pid}")
print(f"🔗 Connect your frontend to: {public_url}")

## 🧪 Test the API

Test the running server to ensure everything works correctly.

In [ ]:
# Test API endpoints
import requests
import time

# Wait for server to start
time.sleep(5)

base_url = str(public_url)

# Test health endpoint
try:
    response = requests.get(f"{base_url}/health")
    print(f"✅ Health check: {response.json()}")
except Exception as e:
    print(f"❌ Health check failed: {e}")

# Test model info endpoint
try:
    response = requests.get(f"{base_url}/model-info")
    print(f"✅ Model info: {response.json()}")
except Exception as e:
    print(f"❌ Model info failed: {e}")

# Test supported labels
try:
    response = requests.get(f"{base_url}/supported-labels")
    data = response.json()
    print(f"✅ Supported labels: {data['total_categories']} categories")
except Exception as e:
    print(f"❌ Supported labels failed: {e}")

## 📊 Monitor Server

Monitor the server status and performance.

In [ ]:
# Monitor server status
import psutil
import GPUtil

def show_system_status():
    print("📊 System Status:")
    
    # CPU usage
    cpu_percent = psutil.cpu_percent(interval=1)
    print(f"   CPU Usage: {cpu_percent}%")
    
    # Memory usage
    memory = psutil.virtual_memory()
    print(f"   RAM Usage: {memory.percent}% ({memory.used/1e9:.1f}GB / {memory.total/1e9:.1f}GB)")
    
    # GPU usage
    if torch.cuda.is_available():
        gpu_memory = torch.cuda.memory_allocated() / 1e9
        gpu_memory_total = torch.cuda.get_device_properties(0).total_memory / 1e9
        print(f"   GPU Memory: {gpu_memory:.1f}GB / {gpu_memory_total:.1f}GB")
    
    # Server process
    if proc.poll() is None:
        print(f"   Server Status: ✅ Running (PID: {proc.pid})")
    else:
        print(f"   Server Status: ❌ Stopped")
    
    print(f"   Public URL: {public_url}")

show_system_status()

In [ ]:
# Server management commands
print("🛠️  Server Management:")
print(f"   Public URL: {public_url}")
print(f"   API Documentation: {public_url}/docs")
print(f"   Server PID: {proc.pid}")
print("")
print("📝 Usage Instructions:")
print("   1. Copy the public URL above")
print("   2. Update your frontend .env file: VITE_API_URL=<public_url>")
print("   3. Start your frontend: npm run dev")
print("   4. Draw doodles and generate art!")
print("")
print("⚠️  Note: The server will stop when this Colab session ends")
print("   Keep this notebook running while using the application")

## 🛑 Stop Server

Run this cell to stop the server when you're done.

In [ ]:
# Stop the server
try:
    proc.terminate()
    proc.wait(timeout=10)
    print("✅ Server stopped gracefully")
except:
    proc.kill()
    print("⚠️  Server force stopped")

# Close ngrok tunnel
ngrok.disconnect(public_url)
print("✅ Ngrok tunnel closed")